# Imports

In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table as Table
import matplotlib.pyplot as plt
import numpy.random as ran
import scipy.optimize as op
import scipy.stats as stat
import emcee
import corner
%matplotlib

Using matplotlib backend: Qt5Agg


# Read in

In [2]:
composite_1 = fits.open("/home/jsm/PycharmProjects/tau_eff/SB99/Fitting/data/stack_1.fits") #128 spectra

model_1 = fits.open("/home/jsm/PycharmProjects/tau_eff/SB99/Fitting/data/model_1.fits") #z~2.441

wave_1 = composite_1[1].data["wavelength"]

flux_1 = composite_1[1].data["flux"]

noise_1 = composite_1[1].data["noise"]

fit_1 = model_1[0].data

z_1 = composite_1[1].header["redshift"]

####################################################

composite_2 = fits.open("/home/jsm/PycharmProjects/tau_eff/SB99/Fitting/data/stack_2.fits") #95 spectra

model_2 = fits.open("/home/jsm/PycharmProjects/tau_eff/SB99/Fitting/data/model_2.fits") #z~2.583

wave_2 = composite_2[1].data["wavelength"]

flux_2 = composite_2[1].data["flux"]

noise_2 = composite_2[1].data["noise"]

fit_2 = model_2[0].data

z_2 = composite_2[1].header["redshift"]

In [3]:
import tau_eff

In [4]:
z,t,terr = tau_eff.lya_opacity(wave_2,flux_2,fit_2,noise_2,z_2)

#c_z = fits.Column(name='REDSHIFT', array=z, unit="z", format="E")

#c_t = fits.Column(name='Tau_Eff', array=t, unit="efective opacity", format="E")

#c_t_sig = fits.Column(name='Tau_Err', array=terr, unit="effective opacity error", format="E")

#t = fits.BinTableHDU.from_columns([c_z, c_t, c_t_sig])

#hdr = t.header

#hdr.set("Z_med", 2.583)

#t.writeto("/home/jsm/PycharmProjects/tau_eff/fits/tau.fits")

In [5]:
plt.errorbar(z,t,yerr=terr,fmt='o')
plt.show()

In [6]:
#the functions

def Tau_ev(z,p0,p1):
    
    return p0*(1+z)**p1

#log space to fit a line!

def lnTau_ev(z,lnp0,p1):
    
    #y  =   m  *   x     + b
    return p1*np.log((1+z)/(1+2.58))+lnp0

In [7]:
x = np.log((1+z)/(1+2.58))

y = np.log(t)

yerr = .434*(terr/t)

In [8]:
#in log space

poly = np.polyfit(x,y,1)

m_true = poly[0]

b_true = poly[1]

xrange = np.linspace(min(x),max(x),100)

plt.errorbar(x,y,yerr=yerr,fmt='o')
plt.plot(xrange,m_true*xrange+b_true)
plt.show()

In [9]:
m_true, b_true

(-0.39641512742906893, -2.0597015042008535)

In [10]:
def lnlike(theta, x, y, yerr):
    m, b = theta
    model = m * x + b
    inv_sigma2 = 1.0/(yerr**2 + model**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [11]:
nll = lambda *args: -lnlike(*args)

result = op.minimize(nll, [m_true, b_true], args=(x, y, yerr))

m_ml, b_ml = result["x"]

In [12]:
def lnprior(theta):
    
    m, b = theta
    
    if -5.0 < m < 5.0 and -8.0 < b < 8.0:
        
        return 0.0
    
    return -np.inf

In [13]:
def lnprob(theta, x, y, yerr):
    
    lp = lnprior(theta)
    
    if not np.isfinite(lp):
        
        return -np.inf
    
    return lp + lnlike(theta, x, y, yerr)

In [14]:
ndim, nwalkers = 2, 100

pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

In [15]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, yerr))

sampler.run_mcmc(pos, 100)

State([[-0.5825306  -1.54976176]
 [ 2.9376825  -1.15738068]
 [ 4.12604674 -0.92728392]
 [ 0.98593628 -1.1167952 ]
 [-0.02825729 -1.20373991]
 [ 4.82783877 -0.9221644 ]
 [ 3.09147013 -1.0298015 ]
 [ 2.26579875 -1.0452404 ]
 [ 1.4372058  -1.12792202]
 [-1.36638821 -1.3887607 ]
 [-1.46479767 -1.27403066]
 [ 0.23097705 -1.32332975]
 [ 1.52765191 -1.0747061 ]
 [-1.60169142 -1.48080285]
 [-1.40212164 -1.46526551]
 [ 0.16754005 -1.27797858]
 [ 3.36845493 -0.82827501]
 [-2.60301878 -1.46817788]
 [ 0.56303684 -1.36701637]
 [-0.79077649 -1.31561267]
 [-4.80486125 -1.63123718]
 [ 0.10354621 -1.15134954]
 [ 1.19178698 -1.10811799]
 [ 4.72673299 -0.84969323]
 [-2.49500604 -1.50105588]
 [-0.78547485 -1.29281165]
 [-4.9075918  -1.67162302]
 [-0.58773401 -1.29672712]
 [-3.27611613 -1.69103499]
 [-1.354829   -1.4318979 ]
 [ 4.49891085 -0.8470796 ]
 [-1.18427243 -1.45754341]
 [ 4.29303417 -0.87960257]
 [-0.86571935 -1.24218204]
 [ 3.85384619 -0.9565506 ]
 [ 1.71350072 -1.14651219]
 [-4.08977017 -1.69828

In [16]:
samples = sampler.chain[:, 50:, :].reshape((-1, ndim))

In [17]:
fig = corner.corner(samples, labels=["$p_1$", "$ln(p_0)$"],
                      truths=[1.34, -1.2])
#fig.savefig("/home/jsm/Research/figures/pspace.png")

In [40]:
plt.errorbar(z,t,yerr=terr,fmt='o')
plt.plot(z,Tau_ev(z,np.exp(-1.2),1.34))
plt.show()

In [18]:
x = np.linspace(0,10,10)

In [24]:
tab = x[:]!=0

In [38]:
hmm = all(x[:]!= 0.0) 